In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
print(os.environ['OPENAI_API_KEY'])

from tests import FakeCustomerChain

from app.main import main_flow, create_task
from app.utils import logger_config
from typing import List
# import time
from app.utils import chain_verbose
# from app.model.system import Conversation, System
from app.model import Message

def get_messages_string(messages: List[Message]):
    result =""
    for msg in messages:
        if msg.role == "user":
            result += f"Customer: {msg.text}\n"
        else:
            result += f"Assistant: {msg.text}\n"
    return result[:-1]

def test_conversation(customer: FakeCustomerChain, first_msg="Hola") -> List[str]:
    
    customer_number="99513718"
    customer_system = System()
    conv = customer_system.get_conversation(customer_number)
    conv.add_user_message(first_msg)

    msg = first_msg
    for i in range(6):
        bnbot_response = main_flow(Message.user_message(msg), customer_number)
        if bnbot_response is None or bnbot_response == "":
            break
        print(bnbot_response)
        conv._add_message(bnbot_response)

        customer_response = customer(get_messages_string(conv.get_messages()))
        if customer_response == "":
            break

        conv.add_user_message(customer_response)

        msg = customer_response
    
    return conv

chain_verbose = True
system = System()


# context_1 = """You are a customer that wants to book an accomodation for the weekend.
# Allways answer in Spanish.
# You ask your requirements one at a time.

# These are your requirements for the accomodation: 
# You are 3 people, you have a dog and you need a garage. The property should be located in Mercedes, Uruguay."""

# conversation = test_conversation(FakeCustomerChain(context=context_1))


# context_2 = """You are a customer that wants to book an accomodation for the weekend.
# Allways answer in Spanish.
# You ask your requirements one at a time.

# You want to book a house for the weekend for 6 people.

# Follow these steps:
# Step 1: Try to book the reservation by providing the necesary info based on your needs.
# Step 2: At the end, you decide not to book because it is too expensive.
# Step 3: If the assistant does not have properties available you thank him.
# """

# conversation = test_conversation(FakeCustomerChain(context=context_2))

context_3 = """You are a customer that wants to book an accomodation for the weekend.
Allways answer in Spanish.
You ask your requirements one at a time.

You want to book a house for the weekend for 6 people.

Follow these steps:
Step 1: Try to book the reservation by providing the necesary info based on your needs.
Step 2: When shown with the options, you decide to change the dates of the reservation for the other weekend.
Step 3: If the assistant does not have properties available you thank him. """

conversation = test_conversation(FakeCustomerChain(context=context_3))

In [ ]:
from app.tools import HouseSelectionAssistantTool

tool = HouseSelectionAssistantTool()
porp = {'Sol': {'property_id': 'Sol', 'name': 'Cabaña "Sol"', 'description': 'Impresionante villa con vistas panorámicas a las montañas. Esta lujosa propiedad ofrece un ambiente tranquilo y relajante con amplios espacios interiores y exteriores. Cuenta con una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. Perfecta para escapadas en familia o con amigos.', 'amenities': ['Wi-Fi', 'private parking', 'pet-friendly', 'barbecue', 'private pool'], 'price': '250.0', 'currency': 'USD', 'max_guests': '8', 'how_to_arrive_instructions': 'El complejo queda en ruta 2 km 287, cerca del pejae. Yendo para Fray Bentos desde Mercedes a mano izquierda. Aqui esta la ubicacion en google maps: https://goo.gl/maps/R8gQZDHVXr2tiPQA8', 'pick_up_keys_instructions': 'Las llaves se encuentran en un box en la puerta de entrada. La clave para abrir el box es 12345.'}}

h = """user: Hola
assistant: Hola, ¿en qué puedo ayudarte?
user: Quiero reservar una casa para el fin de semana para 6 personas.
assistant: ¡Hola! Tenemos una propiedad disponible para las fechas que necesitas. Se llama Cabaña "Sol" y tiene capacidad para 8 personas. Tiene una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. El precio por noche es de 250 USD. ¿Te gustaría reservar esta propiedad?
user: No me gusta esa propiedad, alguna mas barata?"""
tool.run(h, porp)



In [ ]:
import datetime

current_time = datetime.datetime.now()
timestamp_str = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(timestamp_str)
